# Using rpy2 Wrapper with T1D Data

Trying causal analysis on T1D data using the following environments:
1. T1D + FDR
2. T1D + SDR
3. T1D + Ctl
4. T1D + OTHER

## Setup

### Importing libraries

In [ ]:
import numpy as np
import joblib
import pandas as pd

### Importing R wrapper library

In [ ]:
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
from rpy2.robjects import numpy2ri

r = robjects.r
numpy2ri.activate()

### Calling R to import the InvariantCausalPrediction package

In [ ]:
ICP = importr('InvariantCausalPrediction')

## T1D Data

### Defining a function to filter the data (nan) 

In [ ]:
def filter_data(X,y):
    indexes = df['Diabetes'].notna()
    return X[indexes],y[indexes]

### Defining a function to set the target labels

In [ ]:
def define_labels(y):
    labels = np.zeros_like(y,dtype='int')
    labels[y=='T1D'] = 1
    return labels  

### Defining a function to set the environment labels

In [ ]:
def define_indexes(y):
    indexes = [ robjects.vectors.IntVector(np.concatenate((np.where(y=='T1D')[0], np.where(y=='FDR')[0]))+1),
                robjects.vectors.IntVector(np.concatenate((np.where(y=='T1D')[0], np.where(y=='SDR')[0]))+1),
                robjects.vectors.IntVector(np.concatenate((np.where(y=='T1D')[0], np.where(y=='OTHER')[0]))+1),
                robjects.vectors.IntVector(np.concatenate((np.where(y=='T1D')[0], np.where(y=='Ctl')[0]))+1)]
    return indexes

### (Load tiny dataset to check the configuration)

### Load real T1D data

In [ ]:
X = pd.read_csv("design_matrix.csv").to_numpy()

df = pd.read_csv("labels.csv")
dy = df['Diabetes']

X,dy = filter_data(X,dy)
ny = dy.to_numpy()

y = define_labels(ny)
idx = define_indexes(ny)

print('Loaded data')

### Converting into R objects and calling ICP

In [ ]:
rY = robjects.vectors.IntVector(y)
model = ICP.ICP(X,rY,idx)

### Printing the result

In [ ]:
ICP.print_InvariantCausalPrediction(model)

In [ ]:
joblib.dump(model,'model_T1D-R-3.pkl')

### Plotting the result